## Logging 

In [ ]:
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.readers import ExtractiveReader

import logging
import haystack.logging
from haystack import tracing
from haystack.tracing.logging_tracer import LoggingTracer

# Clear any existing handlers from the root logger
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Set up a basic file handler to write logs to 'log.json'
logging.basicConfig(
    level=logging.DEBUG,  # Capture all log levels
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    handlers=[
        logging.FileHandler("log.txt", mode='w'),  # Write logs to 'log.txt'
        logging.StreamHandler()  # Also output to the console for debugging
    ]
)

# Set up Haystack logging configuration
haystack.logging.configure_logging(use_json=True)

# Set Haystack logger level to DEBUG to capture all events
haystack_logger = logging.getLogger("haystack")
haystack_logger.setLevel(logging.DEBUG)

# Tracing setup
tracing.tracer.is_content_tracing_enabled = False  # To enable tracing/logging content (inputs/outputs)
tracing.enable_tracing(
    LoggingTracer(tags_color_strings={"haystack.component.input": "\x1b[1;31m", "haystack.component.name": "\x1b[1;34m"})
)

# Document setup and writing documents to the store
docs = [
    Document(content="Paris is the capital of France."),
    Document(content="Berlin is the capital of Germany."),
    Document(content="Rome is the capital of Italy."),
    Document(content="Madrid is the capital of Spain."),
]
document_store = InMemoryDocumentStore()
document_store.write_documents(docs)

# Adding a warning to verify log output
logging.warning("This is a general warning message.")
haystack_logger.debug("This is a debug message specifically for the Haystack logger.")

# Setting up retriever, reader, and pipeline
retriever = InMemoryBM25Retriever(document_store=document_store)
reader = ExtractiveReader()
reader.warm_up()

extractive_qa_pipeline = Pipeline()
extractive_qa_pipeline.add_component(instance=retriever, name="retriever")
extractive_qa_pipeline.add_component(instance=reader, name="reader")
extractive_qa_pipeline.connect("retriever.documents", "reader.documents")

# Running the pipeline
query = "What is the capital of France?"
extractive_qa_pipeline.run(
    data={"retriever": {"query": query, "top_k": 3}, "reader": {"query": query, "top_k": 2}}
)

print("Log entries saved to log.json")


2024-11-11 13:57:17,582 - WARNING - root - This is a general warning message.
2024-11-11 13:57:17,583 - DEBUG - haystack - This is a debug message specifically for the Haystack logger.


2024-11-11 13:57:17,870 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /deepset/roberta-base-squad2-distilled/resolve/main/config.json HTTP/11" 200 0
2024-11-11 13:57:18,320 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /deepset/roberta-base-squad2-distilled/resolve/main/tokenizer_config.json HTTP/11" 200 0
2024-11-11 13:57:18,505 - DEBUG - haystack.core.pipeline.base - Adding component 'retriever' (<haystack.components.retrievers.in_memory.bm25_retriever.InMemoryBM25Retriever object at 0x000002B6AB1C1090>

Inputs:
  - query: str
  - filters: Optional[Dict[str, Any]]
  - top_k: Optional[int]
  - scale_score: Optional[bool]
Outputs:
  - documents: List[Document])
2024-11-11 13:57:18,508 - DEBUG - haystack.core.pipeline.base - Adding component 'reader' (<haystack.components.readers.extractive.ExtractiveReader object at 0x000002B6A7617A90>

Inputs:
  - query: str
  - documents: List[Document]
  - top_k: Optional[int]
  - score_threshold: Optional

Log entries saved to log.json
